In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-06-15 05:42:50.987557: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 05:42:50.987641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 05:42:50.989462: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/capstone-food/dataset_food.csv


# Dataset Opening

In [2]:
dfdataset = pd.read_csv('/kaggle/input/capstone-food/dataset_food.csv')
dfdataset.head(20)

,merchant_name,category,target_price,price_category,price_min,price_max,alamat,lat,lng,link
0,"330 Kopi, Ciledug",Kopi/Minuman/Roti,20000,Menengah,15000,37000,"330 Kopi, Ciledug, Jakarta",-6.238135,106.716900,https://www.google.com/maps/search/?api=1&quer...
1,"Abe Steak, Cibubur",Aneka nasi/Minuman/Barat,18000,Murah,5000,165000,"Abe Steak, Cibubur, Jakarta",-6.356087,106.879792,https://www.google.com/maps/search/?api=1&quer...
2,"Abuba Steak, Cikarang",Barat,28600,Menengah,13200,352000,"Abuba Steak, Cikarang, Jakarta",-6.194449,106.822920,https://www.google.com/maps/search/?api=1&quer...
3,Aneka Gorengan Siaga,Jajanan/Minuman,5000,Murah,1500,20000,"Aneka Gorengan Siaga, Jakarta",-6.274045,106.839772,https://www.google.com/maps/search/?api=1&quer...
4,Arangsimpur,Chinese,30250,Mahal,6000,48000,"Arangsimpur, Jakarta",-6.240159,106.651125,https://www.google.com/maps/search/?api=1&quer...
5,Ayam Trolok,Aneka nasi/Ayam & bebek/Seafood,6000,Murah,4000,100000,"Ayam Trolok, Jakarta",-6.194449,106.822920,https://www.google.com/maps/search/?api=1&quer...
6,BABAKARAN BUNDA DAFA,Minuman/Jajanan/Martabak,15000,Murah,6000,25000,"BABAKARAN BUNDA DAFA, Jakarta",-6.140373,106.832440,https://www.google.com/maps/search/?api=1&quer...
7,"Bakerman, SCBD",Roti/Minuman/Jajanan,69600,Mahal,12700,626200,"Bakerman, SCBD, Jakarta",-6.228627,106.806919,https://www.google.com/maps/search/?api=1&quer...
8,Bakmi Kun,Ayam & bebek/Bakmie/Cepat saji,27500,Menengah,8000,300000,"Bakmi Kun, Jakarta",-6.270742,106.812494,https://www.google.com/maps/search/?api=1&quer...
9,Bakmi Lopui 93,Bakmie/Chinese/Minuman,30000,Menengah,15000,30000,"Bakmi Lopui 93, Jakarta",-6.152886,106.813341,https://www.google.com/maps/search/?api=1&quer...


# User History

In [3]:
tipe1 = {"category": ['makan'], 'amount':[5000], 'title':['bunga bakery'], 'action':['Expense'], 'food_category':['Roti']}
tipe2 = {"category": ['makan'], 'amount':[7000], 'title':['bangor'], 'action':['Expense'], 'food_category':['Cepat saji']}
tipe3 = {"category": ['makan'], 'amount':[8000], 'title':['miruki'], 'action':['Expense'], 'food_category':['Minuman']}
tipe4 = {"category": ['makan'], 'amount':[9000], 'title':['gorengan'], 'action':['Expense'], 'food_category':['Jajanan']}
tipe5 = {"category": ['makan'], 'amount':[10000], 'title':['Steak'], 'action':['Expense'], 'food_category':['Barat']}
tipe6 = {"category": ['makan'], 'amount':[11000], 'title':['Lamongan'], 'action':['Expense'], 'food_category':['Seafood']}
tipe7 = {"category": ['makan'], 'amount':[15000], 'title':['Sushi'], 'action':['Expense'], 'food_category':['Jepang']}
tipe8 = {"category": ['makan'], 'amount':[16000], 'title':['AYCE'], 'action':['Expense'], 'food_category':['Korea']}
tipe9 = {"category": ['makan'], 'amount':[16000], 'title':['Caffe Latte'], 'action':['Expense'], 'food_category':['Kopi']}
tipe10 = {"category": ['makan'], 'amount':[16000], 'title':['PizzaHut'], 'action':['Expense'], 'food_category':['Pizza & pasta']}
tipe11 = {"category": ['makan'], 'amount':[16000], 'title':['Marhaba'], 'action':['Expense'], 'food_category':['Timur Tengah']}
tipe12 = {"category": ['makan'], 'amount':[16000], 'title':['Mie Baek'], 'action':['Expense'], 'food_category':['Bakmie']}

import pandas as pd
import random

tipes = [tipe1, tipe2, tipe3, tipe4, tipe5, tipe6, tipe7, tipe8, tipe9, tipe10, tipe11, tipe12]

In [4]:

credentials = []
for i in range(1, 101):
    for j in range(1, random.randint(50, 150)):
        cred_data = random.choice(tipes)
        credentials.append({
            "id": i,
            "category": cred_data["category"][0],
            "amount": random.randint(5000, 16000),
            "title": cred_data["title"][0],
            "action": cred_data["action"][0],
            "food_category": cred_data["food_category"][0]
        })

# Create DataFrame
dfhistory = pd.DataFrame(credentials)
dfhistory.head(200)

,id,category,amount,title,action,food_category
0,1,makan,12587,Sushi,Expense,Jepang
1,1,makan,15328,AYCE,Expense,Korea
2,1,makan,15467,Caffe Latte,Expense,Kopi
3,1,makan,12046,Lamongan,Expense,Seafood
4,1,makan,8883,bunga bakery,Expense,Roti
...,...,...,...,...,...,...
195,3,makan,5440,bunga bakery,Expense,Roti
196,3,makan,11806,Steak,Expense,Barat
197,3,makan,8399,bunga bakery,Expense,Roti
198,3,makan,11089,Sushi,Expense,Jepang


In [5]:
dfhistory['id'].describe()

count    10238.000000
mean        50.029010
std         28.072602
min          1.000000
25%         26.000000
50%         50.000000
75%         74.000000
max        100.000000
Name: id, dtype: float64

# Preprocess

## -Dataset

In [6]:
dataset = dfdataset.copy()
dataset = dataset.drop(['target_price', 'price_category', 'price_min', 'price_max', "alamat", 'lat', 'lng', 'link'], axis=1)
dataset = dataset.reset_index(drop=True)
dataset['category'] = dataset['category'].apply(lambda x: x.split('/'))

dataset.head()

,merchant_name,category
0,"330 Kopi, Ciledug","[Kopi, Minuman, Roti]"
1,"Abe Steak, Cibubur","[Aneka nasi, Minuman, Barat]"
2,"Abuba Steak, Cikarang",[Barat]
3,Aneka Gorengan Siaga,"[Jajanan, Minuman]"
4,Arangsimpur,[Chinese]


In [7]:
mlb = MultiLabelBinarizer()
dataset_cats = mlb.fit_transform(dataset['category'])

# Create a DataFrame with one-hot encoded categories
dataset_cat_df = pd.DataFrame(dataset_cats, columns=mlb.classes_, index=dataset['merchant_name'])
print(dataset_cats)
print(dataset_cat_df)

[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
                              Aneka nasi  Ayam & bebek  Bakmie  Bakso & soto  \
merchant_name                                                                  
330 Kopi, Ciledug                      0             0       0             0   
Abe Steak, Cibubur                     1             0       0             0   
Abuba Steak, Cikarang                  0             0       0             0   
Aneka Gorengan Siaga                   0             0       0             0   
Arangsimpur                            0             0       0             0   
...                                  ...           ...     ...           ...   
Xi Bo Ba, Ramayana Cileungsi           0             0       0             0   
Ya'mbok, Jatimelati                    0             1       0             0   
farm.girl, SCBD                        0             0       0             0   


## -History

In [8]:
user = dfhistory.copy()
user = user[user['id'] == 2]
user = user.reset_index(drop=True)
user.head(10)

,id,category,amount,title,action,food_category
0,2,makan,15811,bangor,Expense,Cepat saji
1,2,makan,14967,Caffe Latte,Expense,Kopi
2,2,makan,8691,Caffe Latte,Expense,Kopi
3,2,makan,15925,Mie Baek,Expense,Bakmie
4,2,makan,7979,gorengan,Expense,Jajanan
5,2,makan,12211,gorengan,Expense,Jajanan
6,2,makan,15263,gorengan,Expense,Jajanan
7,2,makan,6547,gorengan,Expense,Jajanan
8,2,makan,14583,gorengan,Expense,Jajanan
9,2,makan,10159,PizzaHut,Expense,Pizza & pasta


In [9]:
history = user.copy()
history = history.drop(["id", "category", "amount", "action"], axis=1)
# history['food_category'] = history['food_category'].apply(lambda x: x.split('/'))
history.head(10)

,title,food_category
0,bangor,Cepat saji
1,Caffe Latte,Kopi
2,Caffe Latte,Kopi
3,Mie Baek,Bakmie
4,gorengan,Jajanan
5,gorengan,Jajanan
6,gorengan,Jajanan
7,gorengan,Jajanan
8,gorengan,Jajanan
9,PizzaHut,Pizza & pasta


In [10]:
user_cats = mlb.transform(history['food_category'].apply(lambda x: [x]))
user_cat_df = pd.DataFrame(user_cats, columns=mlb.classes_)
user_profile = user_cat_df.sum(axis=0)
print(user_cats, '\n', user_cat_df, '\n',user_profile)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 
     Aneka nasi  Ayam & bebek  Bakmie  Bakso & soto  Barat  Cepat saji  \
0            0             0       0             0      0           1   
1            0             0       0             0      0           0   
2            0             0       0             0      0           0   
3            0             0       1             0      0           0   
4            0             0       0             0      0           0   
5            0             0       0             0      0           0   
6            0             0       0             0      0           0   
7            0             0       0             0      0           0   
8            0             0       0             0      0           0   
9            0             0       0             0      0           0   
10           0             0       0             0      0           0   
11

# Model

## -Convert to tensors

In [11]:
dataset_tensor = tf.convert_to_tensor(dataset_cat_df.values, dtype=tf.float32)
user_profile_tensor = tf.convert_to_tensor(user_profile.values, dtype=tf.float32)
user_profile_tensor = tf.expand_dims(user_profile_tensor, axis=0)
user_profile_tensor = tf.tile(user_profile_tensor, [dataset_tensor.shape[0], 1])

## -Define model

In [12]:
input_merchant = tf.keras.Input(shape=(dataset_tensor.shape[1],))
input_user = tf.keras.Input(shape=(user_profile_tensor.shape[1],))

# Cosine similarity calculation
dot_product = tf.keras.layers.Dot(axes=1, normalize=True)([input_merchant, input_user])


model = tf.keras.Model(inputs=[input_merchant, input_user], outputs=dot_product)
model.compile(optimizer='adam', loss='cosine_similarity')

## -Predict

In [13]:
# Calculate similarity
similarity_scores = model.predict([dataset_tensor, user_profile_tensor])

# Create a DataFrame for similarity scores
similarity_df = pd.DataFrame(similarity_scores, index=dataset['merchant_name'], columns=['similarity'])

# Sort merchants by similarity score
recommendations = similarity_df.sort_values(by='similarity', ascending=False)
print(recommendations)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
                              similarity
merchant_name                           
Chateraise, Senayan City        0.631800
Kenduricoffee, Patal Senayan    0.631800
Toko Kopi Bacarita              0.606897
Groen Kopi                      0.606897
Kedai Santai                    0.606897
...                                  ...
Restoran Sederhana, Ampera      0.000000
Restoran Sederhana, Caman       0.000000
Hoghock, Greenlake City         0.000000
Sate Rembiga Racaca             0.000000
martabak mapingkot              0.000000

[221 rows x 1 columns]


## -Evaluate

In [14]:
# Calculate the cosine similarity loss
cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)
loss_value = cosine_loss(user_profile_tensor, dataset_tensor)
loss_value = tf.reduce_mean(loss_value).numpy()

print(f"Cosine Similarity Loss: {loss_value}")

Cosine Similarity Loss: -0.2939484119415283


## -Save model

In [15]:
model.save('recommendation_model.h5')

# Temp